In [1]:
# Import dependencies
import pandas as pd
import json
import re
import concurrent.futures
import time

In [2]:
# Retrieve the geoJSON data which contains the list of country names
with open("static/data/countries.geojson") as f:
    data = json.load(f)
    
    f.close()

In [3]:
# Use list comprehension to retreive the names from the json
names = [i["properties"]["ADMIN"] for i in data["features"]]

In [4]:
# Selecting a country name
country = names[15]
country

'Australia'

In [5]:
# Building the query URL
url = "https://en.wikipedia.org/wiki/"

queryURL = url + country.replace(" ", "_")

queryURL

'https://en.wikipedia.org/wiki/Australia'

In [6]:
# Using pandas to scrape the URL
wiki = pd.read_html(queryURL)[0]
wiki.columns = [1, 2]

In [7]:
# Dropping any N/A values
wiki.dropna(how = "any", inplace = True)
wiki.reset_index(drop = True, inplace = True)

In [8]:
# Cleaning the data
wiki[1] = wiki[1].apply(lambda x: x.replace("•", ""))

In [9]:
# Defining target values & creating an empty list
target = ["Capital", ".+language", "Population", "GDP\xa0(PPP)", "Currency"]
values = []

# Looping through the target values
for i in target:
    # Check if the target is not in the list
    if i not in list(wiki[1]):
        # Checking for the regex expression
        if i == ".+language":
            for x in range(0, len(wiki)):
                # If the regex expression is found, take the first value and end the loop
                if re.search(i, wiki[1][x]):
                    values.append(wiki[2][x])
                    break
        else:
            # If nothing is found, append N/A
            values.append("N/A")
    else:
        # Looping through the dataframe to find data
        for x in range(0, len(wiki)):
            
            if wiki[1][x] == i:
                if wiki[1][x] == "Population":
                    values.append(wiki[2][x + 1])
                    if wiki[1][x + 3] == "GDP\xa0(PPP)":
                        values.append(wiki[2][x + 2])
                    else:
                        values.append(wiki[2][x + 3])
                        
                elif wiki[1][x] == "GDP\xa0(PPP)":
                    values.append(wiki[2][x + 1])
                    values.append(wiki[2][x + 2])
                    
                else:
                    values.append(wiki[2][x])
                    
# Checking the values collected
values

['Canberra35°18′29″S 149°07′28″E\ufeff / \ufeff35.30806°S 149.12444°E',
 'English[N 2]',
 '25,728,000[7] (53rd)',
 '3.3/km2 (8.5/sq\xa0mi) (192nd)',
 '$1.334 trillion[9] (19th)',
 '$51,885[9] (17th)',
 'Australian dollar (AUD)']

In [10]:
# If a capital city is found, clean the value
match = re.match(r"([a-z]+)([0-9]+)", values[0], re.I)
if match:
    items = match.groups()
    values[0] = items[0]

vals = []

# Removing [ values from the data
for x in values:
    vals.append(x.split("[")[0])

vals[3] = vals[3].split(" ")[0]

# Adding the country name
vals.insert(0, country)

vals

['Australia',
 'Canberra',
 'English',
 '25,728,000',
 '3.3/km2',
 '$1.334 trillion',
 '$51,885',
 'Australian dollar (AUD)']

In [11]:
# Creating the titles for the table
titles = ["Country Name: ", "Capital City: ", "Official Language: ", 
          "Population: ", "Population Density: ", "GDP Total: ", "GDP Per Capita: ", "Currency: "]

# Creating a dataframe to be stored
output_df = pd.DataFrame(vals, titles)
output_df

,0
Country Name:,Australia
Capital City:,Canberra
Official Language:,English
Population:,"25,728,000"
Population Density:,3.3/km2
GDP Total:,$1.334 trillion
GDP Per Capita:,"$51,885"
Currency:,Australian dollar (AUD)


In [12]:
# Import dependencies
import pandas as pd
import json
import re
import concurrent.futures
import time

# Retrieve the geoJSON data which contains the list of country names
with open("static/data/countries.geojson") as f:
    data = json.load(f)
    
    f.close()
    
# Use list comprehension to retreive the names from the json
names = [i["properties"]["ADMIN"] for i in data["features"]]

# Building a function which can be used in a loop
def scrape_wiki(country):
    # Building the query URL
    url = "https://en.wikipedia.org/wiki/"
    queryURL = url + country.replace(" ", "_")

    # Using pandas to scrape the URL
    wiki = pd.read_html(queryURL)[0]
    wiki.columns = [1, 2]
    
    # Dropping any N/A values
    wiki.dropna(how = "any", inplace = True)
    wiki.reset_index(drop = True, inplace = True)
    
    # Cleaning the data
    wiki[1] = wiki[1].apply(lambda x: x.replace("•", ""))
    
    # Defining target values & creating an empty list
    target = ["Capital", ".+language", "Population", "GDP\xa0(PPP)", "Currency"]
    values = []

    # Looping through the target values
    for i in target:
        # Check if the target is not in the list
        if i not in list(wiki[1]):
            # Checking for the regex expression
            if i == ".+language":
                for x in range(0, len(wiki)):
                    # If the regex expression is found, take the first value and end the loop
                    if re.search(i, wiki[1][x]):
                        values.append(wiki[2][x])
                        break
            else:
                # If nothing is found, append N/A
                values.append("N/A")
        else:
            # Looping through the dataframe to find data
            for x in range(0, len(wiki)):

                if wiki[1][x] == i:
                    if wiki[1][x] == "Population":
                        values.append(wiki[2][x + 1])
                        if wiki[1][x + 3] == "GDP\xa0(PPP)":
                            values.append(wiki[2][x + 2])
                        else:
                            values.append(wiki[2][x + 3])

                    elif wiki[1][x] == "GDP\xa0(PPP)":
                        values.append(wiki[2][x + 1])
                        values.append(wiki[2][x + 2])

                    else:
                        values.append(wiki[2][x])
                        
    # If a capital city is found, clean the value
    match = re.match(r"([a-z]+)([0-9]+)", values[0], re.I)
    if match:
        items = match.groups()
        values[0] = items[0]

    vals = []

    # Removing [ values from the data
    for x in values:
        vals.append(x.split("[")[0])

    vals[3] = vals[3].split(" ")[0]

    # Adding the country name
    vals.insert(0, country)
    
    # Creating the titles for the table
    titles = ["Country Name: ", "Capital City: ", "Official Language: ", 
              "Population: ", "Population Density: ", "GDP Total: ", "GDP Per Capita: ", "Currency: "]

    # Creating a dataframe to be stored
    df = pd.DataFrame(vals, titles)

    return df

In [13]:
# Creating an empty dictionary to hold all of the country data
d = {}

In [14]:
missed = []

# Creating a function to run the scrape wiki function for each country
def scrape(i):
    try:
        a = scrape_wiki(i)
        d[i] = a
    except:
        missed.append(i)
        
# Creating a function that asynchronously runs the scrape function
def download_results(names):
    threads = 30
    
    with concurrent.futures.ThreadPoolExecutor(max_workers = threads) as executor:
        executor.map(scrape, names)

# Creating a main function to call the previous functions
def main(names):
    t0 = time.time()
    download_results(names)
    t1 = time.time()
    
    print(f"{t1-t0} seconds to download {len(names) - len(missed)} countries.\n{len(missed)} could not be retreived")

In [15]:
# Calling the main function
main(names)

90.51381492614746 seconds to download 202 countries.
53 could not be retreived


In [16]:
import pymongo

In [17]:
# create the connection to MongoDB
# create an instance of the database and collections

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.countries_info_db
countries_info = db.countries_info

In [18]:
# Check dict
d

{'Aruba':                                         0
 Country Name:                       Aruba
 Capital City:                         N/A
 Official Language:        DutchPapiamento
 Population:                       116,576
 Population Density:               624/km2
 GDP Total:                   $4.4 billion
 GDP Per Capita:                   $37,576
 Currency:             Aruban florin (AWG),
 'Aland':                                    0
 Country Name:                  Aland
 Capital City:                    N/A
 Official Language:           Swedish
 Population:                   29,884
 Population Density:        18.36/km2
 GDP Total:            $1.563 billion
 GDP Per Capita:              $55,829
 Currency:             Euro (€) (EUR),
 'Burundi':                                                                      0
 Country Name:                                                  Burundi
 Capital City:                  Gitega (political) Bujumbura (economic)
 Official Language:    K

In [19]:
# From here we need to push this data in dictionary "d" to a database
if (countries_info.find({}).count() == 0):
    countries_info.insert(d)
    
else:
    print("Data already exists")

ipykernel_launcher:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
ipykernel_launcher:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


InvalidDocument: cannot encode object:                                         0
Country Name:                       Aruba
Capital City:                         N/A
Official Language:        DutchPapiamento
Population:                       116,576
Population Density:               624/km2
GDP Total:                   $4.4 billion
GDP Per Capita:                   $37,576
Currency:             Aruban florin (AWG), of type: <class 'pandas.core.frame.DataFrame'>